KNNImputer可以更便捷地处理缺失值，并且与直接用均值、中位数相比更为可靠。利用“近朱者赤”的KNN算法原理，这种插补方法借助其他特征的分布来对目标特征进行缺失值填充。

<br><br>
先来一个小例子开开胃，data中第二个样本存在缺失值。

In [1]:
import numpy as np
from sklearn.impute import KNNImputer

In [7]:
data = [[2, 4, 8], [3, np.nan, 7], [5, 8, 3], [4, 3, 8]]

KNNImputer中的超参数与KNN算法一样，n_neighbors为选择“邻居”样本的个数，先试试n_neighbors=1。

In [8]:
imputer = KNNImputer(n_neighbors=1)
imputer.fit_transform(data)

array([[ 2.,  4.,  8.],
       [ 3.,  4.,  7.],
       [ 5.,  8.,  3.],
       [14.,  3.,  8.]])

可以看到，因为第二个样本的第一列特征3和第三列特征7，与第一行样本的第一列特征2和第三列特征8的欧氏距离最近，所以缺失值按照第一个样本来填充，填充值为4。那么n_neighbors=2呢？

In [9]:
imputer = KNNImputer(n_neighbors=2)
imputer.fit_transform(data)

array([[ 2.,  4.,  8.],
       [ 3.,  6.,  7.],
       [ 5.,  8.,  3.],
       [14.,  3.,  8.]])

此时根据欧氏距离算出最近相邻的是第一行样本与第四行样本，此时的填充值就是这两个样本第二列特征4和3的均值：3.5。

接下来让我们看一个实际案例，该数据集来自Kaggle皮马人糖尿病预测的分类赛题，其中有不少缺失值，我们试试用KNNImputer进行插补。

In [11]:
import numpy as np
import pandas as pd
import ydata_profiling as pp
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context="notebook", style="darkgrid")
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
 
from sklearn.impute import KNNImputer

In [12]:
#Loading the dataset
diabetes_data = pd.read_csv('./data/diabetes.csv')


In [13]:
diabetes_data.columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
                        'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
diabetes_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


在这个数据集中，0值代表的就是缺失值，所以我们需要先将0转化为nan值然后进行缺失值处理。

In [14]:
diabetes_data_copy = diabetes_data.copy(deep=True)
diabetes_data_copy[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = diabetes_data_copy[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0, np.NaN)
 
print(diabetes_data_copy.isnull().sum())

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64


下面用DiabetesPedigreeFunction与Age，对BloodPressure中的35个缺失值进行KNNImputer插补。

In [15]:
null_index = diabetes_data_copy.loc[diabetes_data_copy['BloodPressure'].isnull(), :].index
null_index

Index([  7,  15,  49,  60,  78,  81, 172, 193, 222, 261, 266, 269, 300, 332,
       336, 347, 357, 426, 430, 435, 453, 468, 484, 494, 522, 533, 535, 589,
       601, 604, 619, 643, 697, 703, 706],
      dtype='int64')

In [16]:
imputer = KNNImputer(n_neighbors=10)
diabetes_data_copy[['BloodPressure', 'DiabetesPedigreeFunction', 'Age']] = imputer.fit_transform(diabetes_data_copy[['BloodPressure', 'DiabetesPedigreeFunction', 'Age']])
print(diabetes_data_copy.isnull().sum())

Pregnancies                   0
Glucose                       5
BloodPressure                 0
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64


可以看到现在BloodPressure中的35个缺失值消失了。我们看看具体填充后的数据

In [19]:
diabetes_data_copy.iloc[null_index]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
7,10,115.0,71.1,32.8,NaN,35.3,0.134,29.0,0
15,7,100.0,72.5,29.1,NaN,30.0,0.484,32.0,1
49,7,105.0,69.1,26.2,NaN,NaN,0.305,24.0,0
60,2,84.0,74.9,21.7,NaN,NaN,0.304,21.0,0
78,0,131.0,76.6,32.8,NaN,43.2,0.270,26.0,1
81,2,74.0,63.6,25.7,NaN,NaN,0.102,22.0,0
172,2,87.0,65.6,23.0,NaN,28.9,0.773,25.0,0
193,11,135.0,75.2,34.7,NaN,52.3,0.578,40.0,1
222,7,119.0,77.0,27.8,NaN,25.2,0.209,37.0,0
261,3,141.0,75.9,29.3,NaN,30.0,0.761,27.0,1


到此，BloodPressure中的缺失值已经根据DiabetesPedigreeFunction与Age运用KNNImputer填充完成了。注意的是，对于非数值型特征需要先转换为数值型特征再进行KNNImputer填充操作，因为目前KNNImputer方法只支持数值型特征

In [18]:
imputer = KNNImputer(n_neighbors=10)
diabetes_data_copy[['SkinThickness', 'DiabetesPedigreeFunction', 'Age']] = imputer.fit_transform(diabetes_data_copy[['SkinThickness', 'DiabetesPedigreeFunction', 'Age']])
print(diabetes_data_copy.isnull().sum())

Pregnancies                   0
Glucose                       5
BloodPressure                 0
SkinThickness                 0
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64
